# Convert local pixels to geographical coordinates

This notebook allows to transform the pixels local coordinates of the 50% segments include in polygon to geographical coordinates (latitude, longitude)

In [1]:
import json
import rasterio as rio
from pyproj import Transformer

In [2]:
# Transform the crs of our file in order to convert
path = "./tif_data/"
image_path = path + "aquisition01_band02.tif"
src = rio.open(image_path)
data = src.read(1)
in_epsg = src.crs
out_epsg = "EPSG:4326"
transformer = Transformer.from_crs(in_epsg, out_epsg)

In [3]:
def get_lat_long(pixel0, pixel1):
    """
    This function use the transformer set up above to 
    transform point between the two coordinates systems.

    Args:
        pixel0 (int): coordinate of the pixel in the image
        pixel1 (int): coordinate of the pixel in our image

    Returns:
        lat: latitude coordinate of the pixel
        long: longitude coordinate of the pixel
    """
    coords = src.xy(pixel0, pixel1)
    lat, long = transformer.transform(coords[0], coords[1])
    return lat, long

In [4]:
with open("./data/contained_segments_50.txt", "r") as file:
    lines = file.readlines()

# Initialize a list to store the results
results = []

# Iterate through each line and extract pixel coordinates
for line in lines:
    if "Segment ID" in line:
        # Extracting relevant information from the line
        segment_id = float(line.split("Segment ID: ")[1].split(",")[0])
        polygon_id = float(line.split("Polygon ID: ")[1].split(",")[0])
        class_id = float(line.split("Class ID: ")[1].split(",")[0])

        # Extracting pixel coordinates from the line, handling empty strings
        pixels_raw = line.split("Pixels: ")[1].strip()
        pixels_list = [list(map(int, coord.split(", "))) for coord in pixels_raw[2:-2].split("], [") if coord]

        # Applying the function to each pixel and storing the results
        for pixel in pixels_list:
            lat, long = get_lat_long(pixel[0], pixel[1])
            result = {
                "segment_id": segment_id,
                "polygon_id": polygon_id,
                "class_id": class_id,
                "coordinates": {"latitude": lat, "longitude": long}
            }
            results.append(result)

# Write the results to a JSON file
with open("output.json", "w") as json_file:
    json.dump(results, json_file, indent=2)

print("Results have been written to 'output.json'.")

Results have been written to 'output.json'.
